In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import dask.dataframe as dd
import seaborn as sns
import random
import time
import tensorflow as tf
import math
from scipy import stats
import warnings


import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy import sparse
from sklearn.preprocessing import LabelEncoder


from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm 

from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn import feature_selection
from catboost import CatBoostRegressor

In [2]:
#Code Source : https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [3]:
%%time
print('-' * 80)
print('train')
train_df = import_data('train_df_final.csv')

print('-' * 80)
print('test')
test_df = import_data('test_final_df_V1.csv')

--------------------------------------------------------------------------------
train
Memory usage of dataframe is 13404.53 MB
Memory usage after optimization is: 3111.86 MB
Decreased by 76.8%
--------------------------------------------------------------------------------
test
Memory usage of dataframe is 488.58 MB
Memory usage after optimization is: 115.72 MB
Decreased by 76.3%
CPU times: user 3min 3s, sys: 30.5 s, total: 3min 33s
Wall time: 5min 51s


In [4]:
train_df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,transactions,city,state,type,cluster,family,class,perishable
0,0,2013-01-01,25,103665,7.0,NaN,770.0,Salinas,Santa Elena,D,1,BREAD/BAKERY,2712,1
1,1,2013-01-01,25,105574,1.0,NaN,770.0,Salinas,Santa Elena,D,1,GROCERY I,1045,0
2,2,2013-01-01,25,105575,2.0,NaN,770.0,Salinas,Santa Elena,D,1,GROCERY I,1045,0
3,3,2013-01-01,25,108079,1.0,NaN,770.0,Salinas,Santa Elena,D,1,GROCERY I,1030,0
4,4,2013-01-01,25,108701,1.0,NaN,770.0,Salinas,Santa Elena,D,1,DELI,2644,1


In [5]:
test_df.head()

,id,date,store_nbr,item_nbr,onpromotion,city,state,type_x,cluster,dcoilwtico,type_y,locale,locale_name,description,transferred,family,class,perishable,transactions
0,125497040,2017-08-16,1,96995,0.0,Quito,Pichincha,D,13,46.8125,NaN,NaN,NaN,NaN,NaN,GROCERY I,1093,0,NaN
1,125497041,2017-08-16,1,99197,0.0,Quito,Pichincha,D,13,46.8125,NaN,NaN,NaN,NaN,NaN,GROCERY I,1067,0,NaN
2,125497042,2017-08-16,1,103501,0.0,Quito,Pichincha,D,13,46.8125,NaN,NaN,NaN,NaN,NaN,CLEANING,3008,0,NaN
3,125497043,2017-08-16,1,103520,0.0,Quito,Pichincha,D,13,46.8125,NaN,NaN,NaN,NaN,NaN,GROCERY I,1028,0,NaN
4,125497044,2017-08-16,1,103665,0.0,Quito,Pichincha,D,13,46.8125,NaN,NaN,NaN,NaN,NaN,BREAD/BAKERY,2712,1,NaN


In [6]:
train_df.shape

(125497040, 14)

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 14 columns):
id              int32
date            category
store_nbr       int8
item_nbr        int32
unit_sales      float32
onpromotion     category
transactions    float16
city            category
state           category
type            category
cluster         int8
family          category
class           int16
perishable      int8
dtypes: category(6), float16(1), float32(1), int16(1), int32(2), int8(3)
memory usage: 3.0 GB


In [8]:
train_df.isnull().sum()

id                     0
date                   0
store_nbr              0
item_nbr               0
unit_sales             0
onpromotion     21657651
transactions      214625
city                   0
state                  0
type                   0
cluster                0
family                 0
class                  0
perishable             0
dtype: int64

In [9]:
test_df.shape

(3370464, 19)

In [10]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3370464 entries, 0 to 3370463
Data columns (total 19 columns):
id              int32
date            category
store_nbr       int8
item_nbr        int32
onpromotion     float16
city            category
state           category
type_x          category
cluster         int8
dcoilwtico      float16
type_y          category
locale          category
locale_name     category
description     category
transferred     float16
family          category
class           int16
perishable      int8
transactions    float64
dtypes: category(9), float16(3), float64(1), int16(1), int32(2), int8(3)
memory usage: 115.7 MB


In [11]:
test_df.isnull().sum()

id                    0
date                  0
store_nbr             0
item_nbr              0
onpromotion           0
city                  0
state                 0
type_x                0
cluster               0
dcoilwtico       842616
type_y          3159810
locale          3159810
locale_name     3159810
description     3159810
transferred     3159810
family                0
class                 0
perishable            0
transactions    3370464
dtype: int64

In [12]:
#Dropping Columns from Test Data which had NUll Values and had not included in TRain data as well
test_df.drop(['dcoilwtico', 'type_y','locale','locale_name','description','transferred','transactions'], axis=1, inplace=True)

In [13]:
test_df.shape

(3370464, 12)

In [14]:
test_df.head()

,id,date,store_nbr,item_nbr,onpromotion,city,state,type_x,cluster,family,class,perishable
0,125497040,2017-08-16,1,96995,0.0,Quito,Pichincha,D,13,GROCERY I,1093,0
1,125497041,2017-08-16,1,99197,0.0,Quito,Pichincha,D,13,GROCERY I,1067,0
2,125497042,2017-08-16,1,103501,0.0,Quito,Pichincha,D,13,CLEANING,3008,0
3,125497043,2017-08-16,1,103520,0.0,Quito,Pichincha,D,13,GROCERY I,1028,0
4,125497044,2017-08-16,1,103665,0.0,Quito,Pichincha,D,13,BREAD/BAKERY,2712,1


In [15]:
#Dropping transactions column train since all the transactions in test are NULL 
train_df.drop(['transactions'], axis=1, inplace=True)

In [16]:
train_df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,family,class,perishable
0,0,2013-01-01,25,103665,7.0,NaN,Salinas,Santa Elena,D,1,BREAD/BAKERY,2712,1
1,1,2013-01-01,25,105574,1.0,NaN,Salinas,Santa Elena,D,1,GROCERY I,1045,0
2,2,2013-01-01,25,105575,2.0,NaN,Salinas,Santa Elena,D,1,GROCERY I,1045,0
3,3,2013-01-01,25,108079,1.0,NaN,Salinas,Santa Elena,D,1,GROCERY I,1030,0
4,4,2013-01-01,25,108701,1.0,NaN,Salinas,Santa Elena,D,1,DELI,2644,1


In [17]:
#Replacing True or False with 0,1 values in onpromotion and replacing the NAN values with 0(False)
train_df['onpromotion'] = train_df['onpromotion'].fillna(0)
train_df['onpromotion'] = train_df['onpromotion'].astype(int)

In [18]:
train_df['onpromotion'].value_counts()

0    117686418
1      7810622
Name: onpromotion, dtype: int64

In [19]:
#Replacing all the Unit Sales less than 0 to 0
train_df.unit_sales = np.where(train_df.unit_sales < 0, 0,train_df.unit_sales)

In [20]:
#Log Transforming Unit Sales
#train_df['unit_sales'] = train_df['unit_sales'].apply(lambda x : np.log1p(x))
train_df['unit_sales'] = np.log1p(train_df['unit_sales'])

In [21]:
train_df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,family,class,perishable
0,0,2013-01-01,25,103665,2.079442,0,Salinas,Santa Elena,D,1,BREAD/BAKERY,2712,1
1,1,2013-01-01,25,105574,0.693147,0,Salinas,Santa Elena,D,1,GROCERY I,1045,0
2,2,2013-01-01,25,105575,1.098612,0,Salinas,Santa Elena,D,1,GROCERY I,1045,0
3,3,2013-01-01,25,108079,0.693147,0,Salinas,Santa Elena,D,1,GROCERY I,1030,0
4,4,2013-01-01,25,108701,0.693147,0,Salinas,Santa Elena,D,1,DELI,2644,1


In [22]:
train_df.shape

(125497040, 13)

In [23]:
train_df.isnull().sum()

id             0
date           0
store_nbr      0
item_nbr       0
unit_sales     0
onpromotion    0
city           0
state          0
type           0
cluster        0
family         0
class          0
perishable     0
dtype: int64

In [24]:
#Adding Features like Year,Month & Day of week 
train_df['date'] = pd.to_datetime(train_df['date'])

In [25]:
train_df['year'] = np.uint16(train_df['date'].dt.year)
train_df['month'] = np.uint16(train_df['date'].dt.month)
train_df['day_of_week'] = np.uint8(train_df['date'].dt.dayofweek)

In [26]:
train_df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,family,class,perishable,year,month,day_of_week
0,0,2013-01-01,25,103665,2.079442,0,Salinas,Santa Elena,D,1,BREAD/BAKERY,2712,1,2013,1,1
1,1,2013-01-01,25,105574,0.693147,0,Salinas,Santa Elena,D,1,GROCERY I,1045,0,2013,1,1
2,2,2013-01-01,25,105575,1.098612,0,Salinas,Santa Elena,D,1,GROCERY I,1045,0,2013,1,1
3,3,2013-01-01,25,108079,0.693147,0,Salinas,Santa Elena,D,1,GROCERY I,1030,0,2013,1,1
4,4,2013-01-01,25,108701,0.693147,0,Salinas,Santa Elena,D,1,DELI,2644,1,2013,1,1


In [27]:
#Sorting the df for time based splitting 
train_df = train_df.sort_values(by=['date'], ascending=True)

In [28]:
#Sampling last 50% of the data from main df for Hyperparameter tuning since RAM is crashing 
train_df_50 = train_df.tail(62748520)

In [27]:
train_df_50.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,family,class,perishable,year,month,day_of_week
62708039,62708039,2015-11-21,25,921042,0.693147,0,Salinas,Santa Elena,D,1,PERSONAL CARE,4114,0,2015,11,5
62708029,62708029,2015-11-21,25,913964,1.098612,0,Salinas,Santa Elena,D,1,GROCERY I,1036,0,2015,11,5
62708027,62708027,2015-11-21,25,913363,2.944439,0,Salinas,Santa Elena,D,1,CLEANING,3038,0,2015,11,5
62708026,62708026,2015-11-21,25,911990,1.945910,0,Salinas,Santa Elena,D,1,CLEANING,3038,0,2015,11,5
62708025,62708025,2015-11-21,25,911871,0.693147,0,Salinas,Santa Elena,D,1,CLEANING,3020,0,2015,11,5


In [29]:
train_df_50.shape

(62748520, 16)

In [31]:
train_df_50.to_csv('train_df_50.csv', index=False)

In [7]:
train_df_50 = import_data('train_df_50.csv')

Memory usage of dataframe is 7659.73 MB
Memory usage after optimization is: 1914.94 MB
Decreased by 75.0%


In [30]:
y = train_df_50['unit_sales'].values
x = train_df_50.drop(['unit_sales'], axis=1)
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62748520 entries, 62708039 to 125497039
Data columns (total 15 columns):
id             int32
date           category
store_nbr      int8
item_nbr       int32
onpromotion    int64
city           category
state          category
type           category
cluster        int8
family         category
class          int16
perishable     int8
year           uint16
month          uint16
day_of_week    uint8
dtypes: category(5), int16(1), int32(2), int64(1), int8(3), uint16(2), uint8(1)
memory usage: 2.3 GB


In [31]:
%%time
#Splitting the data into training and cross validation
x_train,x_cv,y_train,y_cv = train_test_split(x,y,test_size=0.2)

print(x_train.shape, y_train.shape)
print(x_cv.shape, y_cv.shape)

(50198816, 15) (50198816,)
(12549704, 15) (12549704,)
CPU times: user 23.9 s, sys: 717 ms, total: 24.7 s
Wall time: 24.6 s


In [32]:
x_train.head(2)

,id,date,store_nbr,item_nbr,onpromotion,city,state,type,cluster,family,class,perishable,year,month,day_of_week
116745784,116745784,2017-05-25,7,1914278,0,Quito,Pichincha,D,8,BEVERAGES,1136,0,2017,5,3
103453193,103453193,2017-01-18,44,661582,1,Quito,Pichincha,A,5,"LIQUOR,WINE,BEER",1330,0,2017,1,2


In [33]:
x_cv.head(2)

,id,date,store_nbr,item_nbr,onpromotion,city,state,type,cluster,family,class,perishable,year,month,day_of_week
89053995,89053995,2016-08-25,38,205209,0,Loja,Loja,D,4,MEATS,2302,1,2016,8,3
68177658,68177658,2016-01-20,8,1473478,1,Quito,Pichincha,D,8,PRODUCE,2012,1,2016,1,2


In [34]:
#Adding Features like Year,Month & Day of week to test data
test_df['date'] = pd.to_datetime(test_df['date'])

In [35]:
test_df['year'] = np.uint16(test_df['date'].dt.year)
test_df['month'] = np.uint16(test_df['date'].dt.month)
test_df['day_of_week'] = np.uint8(test_df['date'].dt.dayofweek)

In [36]:
test_df.head(2)

,id,date,store_nbr,item_nbr,onpromotion,city,state,type_x,cluster,family,class,perishable,year,month,day_of_week
0,125497040,2017-08-16,1,96995,0.0,Quito,Pichincha,D,13,GROCERY I,1093,0,2017,8,2
1,125497041,2017-08-16,1,99197,0.0,Quito,Pichincha,D,13,GROCERY I,1067,0,2017,8,2


In [37]:
#Encoding all the category variables 
encoder = LabelEncoder()

In [38]:
encoder.fit(x['city'])
x_train['city'] = encoder.transform(x_train['city'])
x_cv['city'] = encoder.transform(x_cv['city'])

In [39]:
#Encoding Test data
test_df['city'] = encoder.transform(test_df['city'])

In [40]:
encoder.fit(x_train['state'])
x_train['state'] = encoder.transform(x_train['state'])
x_cv['state'] = encoder.transform(x_cv['state'])

In [41]:
test_df['state'] = encoder.transform(test_df['state'])

In [42]:
encoder.fit(x_train['type'])
x_train['type'] = encoder.transform(x_train['type'])
x_cv['type'] = encoder.transform(x_cv['type'])

In [43]:
test_df['type_x'] = encoder.transform(test_df['type_x'])

In [44]:
encoder.fit(x_train['family'])
x_train['family'] = encoder.transform(x_train['family'])
x_cv['family'] = encoder.transform(x_cv['family'])

In [45]:
test_df['family'] = encoder.transform(test_df['family'])

In [46]:
#Dropping Date Column
x_train.drop(['date'], axis=1, inplace=True)
x_cv.drop(['date'], axis=1, inplace=True)
test_df.drop(['date'], axis=1, inplace=True)

In [47]:
x_train.head(2)

,id,store_nbr,item_nbr,onpromotion,city,state,type,cluster,family,class,perishable,year,month,day_of_week
116745784,116745784,7,1914278,0,18,12,3,8,3,1136,0,2017,5,3
103453193,103453193,44,661582,1,18,12,0,5,22,1330,0,2017,1,2


In [48]:
x_cv.head(2)

,id,store_nbr,item_nbr,onpromotion,city,state,type,cluster,family,class,perishable,year,month,day_of_week
89053995,89053995,38,205209,0,12,8,3,4,24,2302,1,2016,8,3
68177658,68177658,8,1473478,1,18,12,3,8,30,2012,1,2016,1,2


In [49]:
test_df.head(2)

,id,store_nbr,item_nbr,onpromotion,city,state,type_x,cluster,family,class,perishable,year,month,day_of_week
0,125497040,1,96995,0.0,18,12,3,13,12,1093,0,2017,8,2
1,125497041,1,99197,0.0,18,12,3,13,12,1067,0,2017,8,2


### Decision Tree

In [45]:
from tqdm import tqdm 

cv_rmse = []
max_depth = [3,5,7,9,11,15]
for i in tqdm(max_depth):
    dt = DecisionTreeRegressor(max_depth = i)
    dt.fit(x_train, y_train)

    y_cv_pred = dt.predict(x_cv)

    cv_rmse.append(mean_squared_error(y_cv,y_cv_pred,squared=False))
    
    print('RMSE score for max depth ',i,':',mean_squared_error(y_cv,y_cv_pred,squared=False))

 17%|█▋        | 1/6 [02:05<10:29, 125.94s/it]

RMSE score for max depth  3 : 0.8387588649636027


 33%|███▎      | 2/6 [05:36<11:43, 175.77s/it]

RMSE score for max depth  5 : 0.8234657118989652


 50%|█████     | 3/6 [10:33<11:33, 231.26s/it]

...

RMSE score for max depth  9 : 0.7911965430804806


 83%|████████▎ | 5/6 [22:53<05:19, 319.53s/it]

RMSE score for max depth  11 : 0.7721996864724107


100%|██████████| 6/6 [31:15<00:00, 312.65s/it]

RMSE score for max depth  15 : 0.71851333809792


In [51]:
#Best Max Depth is 15
dt_clf = DecisionTreeRegressor(max_depth = 15)
dt_clf.fit(x_train, y_train)

y_test_pred_DT = dt_clf.predict(test_df)

In [52]:
len(y_test_pred_DT)

3370464

In [53]:
y_test_pred_DT[0:10]

array([1.30750164, 1.60819706, 1.55985149, 1.29259813, 1.31102465,
       1.60819706, 1.60819706, 1.60819706, 1.60819706, 1.29259813])

In [54]:
test_df['DT'] = y_test_pred_DT

In [55]:
test_df.head(2)

,id,store_nbr,item_nbr,onpromotion,city,state,type_x,cluster,family,class,perishable,year,month,day_of_week,DT
0,125497040,1,96995,0.0,18,12,3,13,12,1093,0,2017.0,8.0,2.0,1.307502
1,125497041,1,99197,0.0,18,12,3,13,12,1067,0,2017.0,8.0,2.0,1.608197


In [56]:
Final_DT = test_df[['id','DT']]

In [57]:
Final_DT.shape

(3370464, 2)

In [58]:
Final_DT.head(2)

,id,DT
0,125497040,1.307502
1,125497041,1.608197


In [59]:
Final_DT.to_csv('Final_DT_submission.csv', index=False)

In [63]:
test_df.drop(['DT'], axis=1, inplace=True)

### GBDT

In [51]:
import lightgbm as lgb
from lightgbm import LGBMRegressor

In [51]:
from tqdm import tqdm 
cv_rmse = []
n_estimators =  [75,150,500,1000,2000]
for i in tqdm(n_estimators):
    lgbm = LGBMRegressor(n_estimators = i)
    lgbm.fit(x_train, y_train)

    y_cv_pred = lgbm.predict(x_cv)

    cv_rmse.append(mean_squared_error(y_cv,y_cv_pred,squared=False))
    
    print('RMSE score for n_estimators ',i,':',mean_squared_error(y_cv,y_cv_pred,squared=False))

 20%|██        | 1/5 [01:28<05:55, 88.94s/it]

RMSE score for n_estimators  75 : 0.7524774026104797


 40%|████      | 2/5 [03:51<06:01, 120.53s/it]

RMSE score for n_estimators  150 : 0.7276350652930244


 60%|██████    | 3/5 [09:59<07:46, 233.27s/it]

RMSE score for n_estimators  500 : 0.683550898424318


 80%|████████  | 4/5 [20:47<06:37, 397.13s/it]

RMSE score for n_estimators  1000 : 0.6595234394085778


100%|██████████| 5/5 [40:10<00:00, 482.15s/it]

RMSE score for n_estimators  2000 : 0.6370936258375105


In [52]:
print(x_train.shape, y_train.shape)
print(test_df.shape)

(50198816, 14) (50198816,)
(3370464, 14)


In [53]:
#Best n_estimator is 2000
gbdt_clf = LGBMRegressor(n_estimators = 2000)
gbdt_clf.fit(x_train, y_train)

y_test_pred_GBDT = gbdt_clf.predict(test_df)

In [54]:
from sklearn.externals import joblib
joblib.dump(gbdt_clf, 'gbdt_clf.pkl')

['gbdt_clf.pkl']

In [55]:
len(y_test_pred_GBDT)

3370464

In [66]:
y_test_pred_GBDT[:10]

array([0.64258135, 1.01854057, 1.22681567, 1.1421909 , 1.14393394,
       1.81788597, 1.81788597, 1.81788597, 1.81788597, 0.85064498])

In [67]:
test_df['unit_sales'] = y_test_pred_GBDT

In [68]:
test_df.head(2)

,id,store_nbr,item_nbr,onpromotion,city,state,type_x,cluster,family,class,perishable,year,month,day_of_week,unit_sales
0,125497040,1,96995,0.0,18,12,3,13,12,1093,0,2017.0,8.0,2.0,0.642581
1,125497041,1,99197,0.0,18,12,3,13,12,1067,0,2017.0,8.0,2.0,1.018541


In [69]:
Final_GBDT = test_df[['id','unit_sales']]

In [71]:
Final_GBDT.shape

(3370464, 2)

In [70]:
Final_GBDT.head(2)

,id,unit_sales
0,125497040,0.642581
1,125497041,1.018541


In [72]:
Final_GBDT.to_csv('Final_GBDT_submission.csv', index=False)

### Custom Ensemble

In [28]:
#Split the Train Dataset 50-50
D1, D2, y1, y2 = train_test_split(x_train,y_train,test_size=0.5)

print(D1.shape, y1.shape)
print(D2.shape, y2.shape)

(25099408, 14) (25099408,)
(25099408, 14) (25099408,)


In [29]:
x_train = reduce_mem_usage(x_train)
x_cv = reduce_mem_usage(x_cv)
test_df = reduce_mem_usage(test_df)
D1 = reduce_mem_usage(D1)
D2 = reduce_mem_usage(D2)

Memory usage of dataframe is 2776.65 MB
Memory usage after optimization is: 1436.20 MB
Decreased by 48.3%
Memory usage of dataframe is 694.16 MB
Memory usage after optimization is: 359.05 MB
Decreased by 48.3%
Memory usage of dataframe is 167.15 MB
Memory usage after optimization is: 80.36 MB
Decreased by 51.9%
Memory usage of dataframe is 1388.33 MB
Memory usage after optimization is: 718.10 MB
Decreased by 48.3%
Memory usage of dataframe is 1388.33 MB
Memory usage after optimization is: 718.10 MB
Decreased by 48.3%


In [55]:
#Sampling with replacement
def generating_samples(input_data, target_data):

    '''Code for generating k samples '''

    selecting_rows = np.random.choice(list(range(input_data.shape[0])),
                                      size=int(0.5 * len(input_data)),
                                      replace=True)

    sample_data = input_data.iloc[selecting_rows]
    
    target_of_sample_data = target_data[selecting_rows]
   
        
    return sample_data,target_of_sample_data

In [56]:
list_input_data =[]
list_output_data =[]

for i in tqdm(range(14)):
    a,b = generating_samples(D1,y1)
    list_input_data.append(a)
    list_output_data.append(b)

100%|██████████| 14/14 [01:24<00:00,  6.02s/it]


In [57]:
list_input_data[0]

,id,store_nbr,item_nbr,onpromotion,city,state,type,cluster,family,class,perishable,year,month,day_of_week
96597293,96597293,31,255199,0,1,9,1,10,12,1044,0,2016.0,11.0,4.0
92002930,92002930,22,1229591,0,16,11,2,7,25,4118,0,2016.0,9.0,6.0
71134210,71134210,37,1919795,0,3,0,3,2,8,2116,1,2016.0,2.0,5.0
76121828,76121828,47,898656,0,18,12,0,14,8,2116,1,2016.0,4.0,1.0
104355100,104355100,35,1489835,0,15,6,2,3,30,2016,1,2017.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83247131,83247131,30,1958185,0,8,6,2,3,3,1142,0,2016.0,6.0,6.0
75199266,75199266,15,1012787,0,9,7,2,15,12,1012,0,2016.0,4.0,6.0
68297246,68297246,28,464374,0,8,6,4,10,3,1124,0,2016.0,1.0,3.0
117100665,117100665,25,1463885,0,20,13,3,1,3,1144,0,2017.0,5.0,6.0


In [58]:
list_output_data[0]

array([1.3862944, 0.6931472, 2.3025851, ..., 0.6931472, 2.3978953,
       1.0184863], dtype=float32)

In [59]:
#Trained base models (DT Regressor) for each sample
models_lst = []
for i in tqdm(range(14)):
    DT = DecisionTreeRegressor()
    DT.fit(list_input_data[i],list_output_data[i])
    models_lst.append(DT)

100%|██████████| 14/14 [42:27<00:00, 181.98s/it]


In [60]:
len(models_lst)

14

In [61]:
#pass the D2 dataset to these models to get n predictions
predictions = []
for model in tqdm(models_lst):
    y_pred = model.predict(D2)   
    predictions.append(y_pred)

100%|██████████| 14/14 [07:28<00:00, 32.03s/it]


In [62]:
len(predictions)

14

In [63]:
predictions[0]

array([0.69314718, 2.19722462, 1.60943794, ..., 2.99573231, 1.60943794,
       1.09861231])

In [64]:
#Create meta dataset
def create_metadata(predicted):
    
    meta_dict = {}
    for i in range(len(predicted)):
        meta_dict['f'+str(i)] = predicted[i]
    ensemble = pd.DataFrame(meta_dict)
    return ensemble

In [65]:
D_meta = create_metadata(predictions)

In [66]:
type(D_meta)

pandas.core.frame.DataFrame

In [67]:
D_meta.shape

(25099408, 14)

In [68]:
D_meta.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
0,0.693147,1.609438,1.791759,1.609438,1.609438,1.609438,1.609438,1.609438,0.693147,0.693147,1.609438,1.609438,1.609438,0.693147
1,2.197225,2.197225,1.945910,1.609438,1.791759,1.098612,2.197225,1.791759,1.791759,2.197225,2.197225,3.610918,3.091043,1.945910
2,1.609438,1.945910,1.098612,1.098612,0.693147,1.609438,1.609438,1.098612,1.098612,1.791759,1.098612,1.098612,1.098612,0.693147
3,1.791759,2.197225,1.609438,1.609438,1.609438,2.197225,1.386294,0.693147,1.791759,1.386294,1.609438,1.791759,1.791759,1.609438
4,1.098612,0.693147,1.098612,0.693147,1.098612,1.386294,1.386294,1.098612,0.693147,1.098612,0.693147,1.386294,1.098612,0.693147


In [69]:
#Now inorder to measure the performance of this entire ensemble model, we will use the 20% test set which we created in the first step
#First pass X_test to each of the base models to get n predictions

x_test_predictions = []
for model in tqdm(models_lst):
    y_pred = model.predict(x_cv)   
    x_test_predictions.append(y_pred)

100%|██████████| 14/14 [03:44<00:00, 16.03s/it]


In [70]:
#Create meta dataset
def create_metadata(predicted):
    
    meta_dict = {}
    for i in range(len(predicted)):
        meta_dict['f'+str(i)] = predicted[i]
    ensemble = pd.DataFrame(meta_dict)
    return ensemble

In [71]:
D_test_meta = create_metadata(x_test_predictions)
print(D_test_meta.shape)

(12549704, 14)


In [72]:
D_test_meta.head(2)

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
0,2.079442,1.386294,1.94591,1.098612,1.609438,1.386294,1.098612,1.098612,0.693147,1.386294,0.693147,0.693147,0.693147,0.693147
1,0.693147,0.693147,1.94591,0.693147,1.098612,0.693147,1.945910,1.098612,0.693147,0.693147,0.693147,0.693147,1.386294,1.609438


In [31]:
print(D_meta.shape,y2.shape)
print(D_test_meta.shape,y_cv.shape)

(25099408, 14) (25099408,)
(12549704, 14) (12549704,)


In [74]:
D_meta.to_csv('D_meta.csv', index=False)

In [75]:
D_test_meta.to_csv('D_test_meta.csv',index=False)

In [30]:
D_meta = import_data('D_meta.csv')
D_test_meta = import_data('D_test_meta.csv')

Memory usage of dataframe is 2680.91 MB
Memory usage after optimization is: 670.23 MB
Decreased by 75.0%
Memory usage of dataframe is 1340.45 MB
Memory usage after optimization is: 335.11 MB
Decreased by 75.0%


In [40]:
#Hyperparameter tuning
import lightgbm as lgb
from lightgbm import LGBMRegressor
from tqdm import tqdm 

cv_rmse = []
n_estimators =  [1000,2000]
for i in tqdm(n_estimators):
    ensemble = LGBMRegressor(n_estimators = i)
    ensemble.fit(D_meta, y2) #fit a meta model on the Train Meta Data created 

    y_cv_pred = ensemble.predict(D_test_meta) #Predicting on the Test Meta Data created

    cv_rmse.append(mean_squared_error(y_cv,y_cv_pred,squared=False))
    
    print('RMSE score for n_estimators ',i,':',mean_squared_error(y_cv,y_cv_pred,squared=False))

 50%|█████     | 1/2 [04:35<04:35, 275.62s/it]

RMSE score for n_estimators  1000 : 0.8743847426808965


100%|██████████| 2/2 [18:04<00:00, 542.07s/it]

RMSE score for n_estimators  2000 : 0.8744497334157538


In [77]:
len(y_cv_pred)

12549704

In [34]:
#Best n_estimator is 2000
ensemble_clf = LGBMRegressor(n_estimators = 2000)
ensemble_clf.fit(D_meta, y2)

#Predicting on Test Data
y_test_pred_ensemble = ensemble_clf.predict(test_df)

In [35]:
y_test_pred_ensemble.shape

(3370464,)

In [36]:
test_df['unit_sales'] = y_test_pred_ensemble

In [37]:
Final_Ensemble = test_df[['id','unit_sales']]

In [38]:
Final_Ensemble.shape

(3370464, 2)

In [39]:
Final_Ensemble.to_csv('Final_Ensemble_submission.csv', index=False)

In [42]:
pip install prettytable

Note: you may need to restart the kernel to use updated packages.


In [43]:
#Code Source: zetcode.com/python/prettytable/
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Model", "Hyper Parameter:max_depth,n_estimators", "RMSE"]
x.add_row(["DT", 15, 0.71])
x.add_row(["GBDT", 2000, 0.63])
x.add_row(["Custom Ensemble", 2000, 0.87])
print(x)

+-----------------+----------------------------------------+------+
|      Model      | Hyper Parameter:max_depth,n_estimators | RMSE |
+-----------------+----------------------------------------+------+
|        DT       |                   15                   | 0.71 |
|       GBDT      |                  2000                  | 0.63 |
| Custom Ensemble |                  2000                  | 0.87 |
+-----------------+----------------------------------------+------+
